In [1]:
import cv2
import numpy as np
import glob
import random
import os
import shutil

In [2]:
net = cv2.dnn.readNet("newspaper_3000.weights", "yolov3_testing.cfg")
classes = ["ROI"]

In [19]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

folder="OCR Samples/"
img_path=random.choice(os.listdir(folder))

img_path=folder+img_path
print(img_path)

# final_image_name=img_path.split("\\")[4]
img = cv2.imread(img_path)
cv2.imwrite(".\input_f.jpg",img)

# original img = cv2.resize(img, None, fx=0.4, fy=0.4)
img = cv2.resize(img, None, fx=0.9, fy=0.9)
height, width, channels = img.shape

# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        # orignal if confidence > 0.5:
        if confidence > 0.6:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)
# if len(indexes)==0:
#     cv2.imwrite(nowatermarkimages+final_image_name,img)
#     print(f"{nowatermarkimages+final_image_name}------ NO WATERMARK")
# else:
#     cv2.imwrite(watermarkimages+final_image_name,img)
#     print(f"{nowatermarkimages+final_image_name}------ WATERMARK")
count=0
region_json={}
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        region_json[i]=[x,y,w,h]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,255), 4)
        count=count+1

        #cv2.putText(img, str(float(confidences[0])), (x, y + 30), font, 3, color, 2)
cv2.imwrite(f".\output_f.jpg",img)
img = cv2.resize(img, (900, 600))
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.destroyAllWindows()    
print("---------------------Region points--------------------")
print(region_json)

OCR Samples/page_0001 (35).jpg
---------------------Region points--------------------
{1: [124, 545, 333, 726], 4: [2166, 571, 404, 493], 5: [1741, 561, 420, 1070], 11: [2583, 573, 408, 1122], 12: [481, 557, 436, 1644], 13: [917, 558, 415, 1458], 14: [1335, 559, 403, 1608], 16: [73, 1307, 432, 1072], 17: [2574, 1696, 416, 493], 19: [1350, 2109, 380, 423], 20: [1760, 1569, 384, 1392], 24: [908, 1982, 428, 1043], 25: [2161, 1027, 409, 2794], 26: [501, 2209, 414, 906], 28: [2561, 2208, 441, 810], 29: [63, 2399, 422, 1510], 32: [1331, 2574, 420, 1333], 34: [1750, 2918, 401, 611], 38: [912, 3008, 414, 951], 40: [2569, 3021, 431, 806], 41: [497, 3115, 425, 812], 43: [1748, 3541, 388, 401]}


In [5]:
#THANK YOU!!!! 
